In [2]:
import math
import random
import numpy as np

In [3]:
# Lmax: Retransmit limit
# TXOP: in ms
# Imax: total slots in worst case
def simulation(sta_amount, RA_RU, Lmax, TXOP, ocw_min, ocw_max, mode='general'):
    total_power = 0
    counter_delay_slot=0
    success_sta=0
    # setting ocw_n [ocw_min, 1st_retry_ocw....]
    ocw = [0 for _ in range(0, Lmax)]
    ocw[0] = ocw_min
    for n in range (1, Lmax):
        temp_ocw = ocw[n-1] * 2 + 1
        if temp_ocw <= ocw_max:
            ocw[n] = temp_ocw
        else:
            ocw[n] = ocw[n-1]
    # Imax
    Imax = 0
    for n in range(0, Lmax):
        Imax += math.ceil(ocw[n] / RA_RU)
    # ------------------------------------ STA setting ------------------------------------------
    # set each STA's OBO counter
    obo_counter = [random.randint(0, ocw_min) for _ in range(0, sta_amount)]
    # set STA's transmit attempt time
    retrans_counter = [0 for _ in range(0, sta_amount)]
#     # selected RA-RU index for each station
#     sta_selected_R = [None for _ in range(0, sta_amount)]
    # ------------------------------------ STA setting ------------------------------------------
    # number of STAs on RA-RU
    sta_on_R = [[[] for _ in range(0,RA_RU)] for _ in range(0,Imax)]
    
    success_info = []
    # No.i slot    
    for i in range(0, Imax):
        # No.x STA
        for x in range (0,sta_amount):
            # decrease OBO counter by RA-RU number
            obo_counter[x] -= RA_RU
            # if counter <=0 select R
            if (obo_counter[x] <= 0 and retrans_counter[x] < 5):
                selected_R_index = random.randint(0, RA_RU-1)
                sta_on_R[i][selected_R_index].append(x)
        # check the balls in R
        for r in range(0, len(sta_on_R[i])):
            # Record power (no power control so always 1)
            total_power += len(sta_on_R[i][r]) * 1
            
            # success STA
            if (len(sta_on_R[i][r]) == 1):
                success_sta_index = sta_on_R[i][r][0]
                # mark 99999 = won't trasnmit again
                obo_counter[success_sta_index] = 99999
                # record
                counter_delay_slot += (i+1)
                success_sta += 1
                # no power control so always 1
#                 total_power += 1 * (retrans_counter[success_sta_index] + 1)
            else:
                for fail_idx in sta_on_R[i][r]:
                    retrans_counter[fail_idx] += 1
                    if (retrans_counter[fail_idx] < 5):
                        obo_counter[fail_idx] = random.randint(0, ocw[retrans_counter[fail_idx]])
#     print(counter_delay_slot)
#     print(success_sta)
    success_probability = success_sta / sta_amount
    access_delay = 0
    if success_sta > 0:
        access_delay = (TXOP * counter_delay_slot) / success_sta
    
    through_put = success_sta / Imax
    utilization = success_sta / (Imax * RA_RU)
    
    average_power = 0
    if success_sta > 0:
        average_power = total_power / success_sta
#     average_power = total_power / sta_amount
    
    return success_probability, access_delay, through_put, utilization, average_power

In [7]:
sample = 1000

# 1, 2, ... 10, 20, 30, ... 100, 200, 300, ... 1000
m_list_simu = np.concatenate([np.arange(1, 11, 1), np.arange(20, 110, 10), np.arange(200, 1100, 100)])
Ps_list_simu = [0 for i in range(28)]
Da_list_simu = [0 for i in range(28)]
T_list_simu = [0 for i in range(28)]
U_list_simu = [0 for i in range(28)]
power_list_simu = [0 for i in range(28)]

# m_list_simu = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]
# Ps_list_simu = [0 for i in range(20)]
# Da_list_simu = [0 for i in range(20)]
# T_list_simu = [0 for i in range(20)]
# U_list_simu = [0 for i in range(20)]

Lmax=5
Lmax=5
TXOP=5.673
ocw_min=7
ocw_max=31

R_list = [9,18,37,74]
for R in R_list:
    for idx, m in enumerate(m_list_simu):
        success_probability_total = 0;
        access_delay_total = 0;
        through_put_total = 0;
        utilization_total = 0;
        power_usage_total = 0;
        no_success_counter = 0;
        for i in range(sample):
            success_probability, access_delay, through_put, utilization, power_usage = simulation(sta_amount=m, RA_RU=R, Lmax=Lmax, TXOP=TXOP, ocw_min=ocw_min, ocw_max=ocw_max, mode='general')
            success_probability_total += success_probability
            access_delay_total += access_delay
            through_put_total += through_put
            utilization_total += utilization
            power_usage_total += power_usage

        Ps = success_probability_total / sample
        Da = access_delay_total / sample
        T = through_put_total / sample
        U = utilization_total / sample
#         power = power_usage_total / (sample - no_success_counter)
        if (sample - no_success_counter) > 0:
            power = power_usage_total / (sample - no_success_counter)

        Ps_list_simu[idx] = Ps
        Da_list_simu[idx] = Da
        T_list_simu[idx] = T
        U_list_simu[idx] = U
        power_list_simu[idx] = power

    print(f'ps_list_simu_{R} = {Ps_list_simu}')
    print(f'da_list_simu_{R} = {Da_list_simu}')
    print(f't_list_simu_{R} = {T_list_simu}')
    print(f'u_list_simu_{R} = {U_list_simu}')
    print(f'power_list_simu_{R} = {power_list_simu} \n')

ps_list_simu_9 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999, 0.9907000000000015, 0.8987999999999963, 0.7080750000000002, 0.51548, 0.3758166666666672, 0.28052857142857157, 0.21373750000000008, 0.16980000000000017, 0.13877999999999996, 0.04502999999999989, 0.02604666666666674, 0.018245000000000008, 0.013443999999999932, 0.010518333333333357, 0.008691428571428557, 0.007161249999999996, 0.006212222222222209, 0.005428999999999958]
da_list_simu_9 = [5.672999999999856, 6.634573499999852, 7.60938399999985, 8.21166749999989, 8.972416799999973, 9.900330500000003, 10.608509999999956, 11.835296250000034, 12.763619666666619, 13.441417100000065, 23.598876861661527, 32.95011113396861, 39.24632899070042, 43.78672304030339, 47.70942393496724, 50.967336961077095, 54.12554206017636, 56.74177662186379, 59.125715218053244, 69.72547383308294, 72.56896006657536, 74.14072126396113, 75.01151058057782, 76.1694718453381, 76.66239015061623, 77.31111906331198, 77.69300190073312, 78.1918126104982]
t_list_